# Create Agents to Research and Write an Article
In this lesson, you will be introduced to the foundational concepts of multi-agent systems and get an overview of the crewAI framework.

In [1]:
# !pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29

ERROR: Ignored the following versions that require a different python version: 0.14.0 Requires-Python >=3.10,<=3.13; 0.14.0rc0 Requires-Python >=3.10,<3.12; 0.14.0rc1 Requires-Python >=3.10,<=3.13; 0.14.1 Requires-Python >=3.10,<=3.13; 0.14.3 Requires-Python >=3.10,<=3.13; 0.14.4 Requires-Python >=3.10,<=3.13; 0.16.0 Requires-Python >=3.10,<=3.13; 0.16.1 Requires-Python >=3.10,<=3.13; 0.16.2 Requires-Python >=3.10,<=3.13; 0.16.3 Requires-Python >=3.10,<=3.13; 0.17.0 Requires-Python >=3.10,<=3.13; 0.17.0rc0 Requires-Python >=3.10,<=3.13; 0.17.0rc1 Requires-Python >=3.10,<=3.13; 0.17.0rc2 Requires-Python >=3.10,<=3.13; 0.19.0 Requires-Python >=3.10,<=3.13; 0.22.0 Requires-Python >=3.10,<=3.13; 0.22.2 Requires-Python >=3.10,<=3.13; 0.22.3 Requires-Python >=3.10,<=3.13; 0.22.4 Requires-Python >=3.10,<=3.13; 0.22.5 Requires-Python >=3.10,<=3.13; 0.27.0 Requires-Python <=3.13,>=3.10; 0.27.0rc0 Requires-Python <=3.13,>=3.10; 0.27.0rc1 Requires-Python <=3.13,>=3.10; 0.27.0rc2 Requires-Python <

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [2]:
from crewai import Agent, Task, Crew

In [8]:
import os
from dotenv import load_dotenv
# from utils import get_openai_api_key, pretty_print_result
# from utils import get_serper_api_key

# Load the environment variables
load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')
os.environ['SERPER_API_KEY'] = os.getenv('SERPER_API_KEY')
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'

## Creating Agents
* Define your Agents, and provide them a role, goal and backstory.
* It has been seen that LLMs perform better when they are role playing.

#### Agent: Planner
Note: The benefit of using multiple strings :

varname = "line 1 of text"
          "line 2 of text"
versus the triple quote docstring:

varname = """line 1 of text
             line 2 of text
          """
is that it can avoid adding those whitespaces and newline characters, making it better formatted to be passed to the LLM.

In [9]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    allow_delegation=False,
	verbose=True
)

LLM value is None


#### Agent: Writer

In [10]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    verbose=True
)

LLM value is None


#### Agent: Editor

In [11]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    verbose=True
)

LLM value is None


## Creating Tasks
Define your Tasks, and provide them a description, expected_output and agent.
#### Task: Plan

In [12]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)

#### Task: Write

In [13]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
)

#### Task: Edit

In [14]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

## Creating the Crew
* Create your crew of Agents
* Pass the tasks to be performed by those agents.
> Note: For this simple example, the tasks will be performed sequentially (i.e they are dependent on each other), so the order of the task in the list matters.
* verbose=2 allows you to see all the logs of the execution.

In [16]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=True
)

## Running the Crew
Note: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [17]:
result = crew.kickoff(inputs={"topic": "Artificial Intelligence"})

# Agent: Content Planner
## Task: 1. Prioritize the latest trends, key players, and noteworthy news on Artificial Intelligence.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


# Agent: Content Planner
## Final Answer: 
Content Plan Document:

Title: Unleashing the Power of Artificial Intelligence: Exploring Trends, Key Players, and Future Implications

Outline:
I. Introduction
    A. Definition of Artificial Intelligence (AI)
    B. Importance and applications of AI in various industries
II. Latest Trends in AI
    A. Machine learning advancements
    B. Ethical considerations in AI development
    C. Rise of AI in healthcare and finance sectors
III. Key Players in the AI Industry
    A. Technology giants (Google, Microsoft, etc.)
    B. Startups making innovative contributions
IV. Noteworthy News in AI


In [20]:
print(result)

# Unleashing the Power of Artificial Intelligence: Exploring Trends, Key Players, and Future Implications

## Introduction
Artificial Intelligence (AI) is reshaping industries worldwide by emulating human intelligence through algorithms and data processing. Its applications range from personalized recommendations in e-commerce to driving autonomous vehicles, demonstrating its significance across diverse sectors. The ability of AI to swiftly analyze vast data sets, identify patterns, and make autonomous decisions is driving efficiency and fostering innovation.

## Latest Trends in AI
The landscape of AI is continuously evolving, predominantly led by advancements in machine learning. Machine learning, a subset of AI, revolves around creating algorithms that enable systems to learn and enhance performance without explicit programming. The ethical dimensions of AI development are gaining prominence, underscoring the importance of transparency and accountability in the utilization of AI tec

In [22]:
from IPython.display import Markdown
Markdown(result.raw)

# Unleashing the Power of Artificial Intelligence: Exploring Trends, Key Players, and Future Implications

## Introduction
Artificial Intelligence (AI) is reshaping industries worldwide by emulating human intelligence through algorithms and data processing. Its applications range from personalized recommendations in e-commerce to driving autonomous vehicles, demonstrating its significance across diverse sectors. The ability of AI to swiftly analyze vast data sets, identify patterns, and make autonomous decisions is driving efficiency and fostering innovation.

## Latest Trends in AI
The landscape of AI is continuously evolving, predominantly led by advancements in machine learning. Machine learning, a subset of AI, revolves around creating algorithms that enable systems to learn and enhance performance without explicit programming. The ethical dimensions of AI development are gaining prominence, underscoring the importance of transparency and accountability in the utilization of AI technologies. Notably, the healthcare and finance industries are experiencing a surge in AI applications, from predictive analytics in patient care to algorithmic trading in financial markets.

## Key Players in the AI Industry
Major players in the tech sphere such as Google, Microsoft, and Amazon are spearheading AI research and development, channeling substantial investments into cutting-edge technologies like natural language processing and computer vision. Moreover, startups are injecting innovation into the AI domain, offering novel perspectives and agile solutions to intricate problems. This collaboration and competition among key players create a dynamic environment propelling the rapid evolution of AI technologies.

## Noteworthy News in AI
Recent milestones in AI research, including the advancement of self-learning algorithms and generative adversarial networks, are garnering attention from academia and industry alike. AI integrations are revolutionizing sectors spanning education to cybersecurity, unlocking fresh avenues for automation and optimization. The continuous advancement in AI holds the promise of reshaping our lifestyles and work dynamics, unlocking opportunities for heightened productivity and effective solutions.

## Target Audience Analysis
This exploration of AI primarily caters to professionals in the tech sector, business executives seeking innovation, and students intrigued by future trends. Their mutual interest in technological progress and deciphering intricate AI concepts underscores the necessity for comprehensive insights into the prevailing trends and major players in the AI realm. By addressing their concerns and aligning with their interests, this article strives to provide valuable perspectives into the dynamic realm of AI.

In conclusion, as AI broadens its horizons and capabilities, staying abreast of industry trends and advancements becomes imperative for professionals and enthusiasts alike. By delving into the latest developments, prominent entities, and future ramifications of AI, we can better comprehend the transformative potential of this potent technology. Embracing the opportunities presented by AI, while navigating ethical considerations, will be pivotal in harnessing its full potential for societal betterment.

In [ ]:
topic = "YOUR TOPIC HERE"
result = crew.kickoff(inputs={"topic": topic})

In [ ]:
Markdown(result)

## Other Popular Models as LLM for your Agents¶

In [ ]:
# Hugging Face (HuggingFaceHub endpoint)
from langchain_community.llms import HuggingFaceHub

llm = HuggingFaceHub(
    repo_id="HuggingFaceH4/zephyr-7b-beta",
    huggingfacehub_api_token="<HF_TOKEN_HERE>",
    task="text-generation",
)

### you will pass "llm" to your agent function